# Topic Modeling 

Topic modeling is a statistical model to discover the abstract "topics" that occur in a collection of documents.  
It is commonly used in text document. But nowadays, in social media analysis, topic modeling is an emerging research area.  
One of the most popular algorithms used is Latent Dirichlet Allocation which was proposed by  
[David Blei et al in 2003](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf).   
Here, I want to perform topic modeling for the upvoted kaggle dataset. 

Some notes on topic modeling:   
* To determine the number topics, it is common to use [elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering) with [perplexity score](http://qpleple.com/perplexity-to-evaluate-topic-models/) as its cost function.   
* To evaluate the models, we can calculate [topic coherence](http://qpleple.com/topic-coherence-to-evaluate-topic-models/).   
* Finally, to interpret the topics, as studied in social science research, there is [triangulation method](http://www.federica.eu/users/9/docs/amaturo-39571-01-Triangulation.pdf).  

## Import libraries

I used LDA model from gensim. Other option is using sklearn.

In [11]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import pandas as pd
import gensim #statistical model lib in NLP
import pyLDAvis.gensim #Visualization lib


## Initiating Tokenizer and Lemmatizer

Initiate the tokenizer, stop words, and lemmatizer from the libraries.

* Tokenizer is used to split the sentences into words.  
* Lemmatizer (a quite similar term to Stemmer) is used to reduce words to its base form.   
The simple difference is that Lemmatizer considers the meaning while Stemmer does not. 


In [12]:
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
en_stop = get_stop_words('en')
lemmatizer = WordNetLemmatizer()

In [13]:
remove_words = ['data','dataset','datasets','content','context','acknowledgement','inspiration']

## Read the data

In [14]:
# Input from csv
df = pd.read_csv('../input/voted-kaggle-dataset.csv')

# sample data
print(df['Description'].head(2))

0    The datasets contains transactions made by cre...
1    The ultimate Soccer database for data analysis...
Name: Description, dtype: object


In [15]:
df.head()

,Title,Subtitle,Owner,Votes,Versions,Tags,Data Type,Size,License,Views,Download,Kernels,Topics,URL,Description
0,Credit Card Fraud Detection,Anonymized credit card transactions labeled as...,Machine Learning Group - ULB,1241,"Version 2,2016-11-05|Version 1,2016-11-03",crime\nfinance,CSV,144 MB,ODbL,"442,136 views","53,128 downloads","1,782 kernels",26 topics,https://www.kaggle.com/mlg-ulb/creditcardfraud,The datasets contains transactions made by cre...
1,European Soccer Database,"25k+ matches, players & teams attributes for E...",Hugo Mathien,1046,"Version 10,2016-10-24|Version 9,2016-10-24|Ver...",association football\neurope,SQLite,299 MB,ODbL,"396,214 views","46,367 downloads","1,459 kernels",75 topics,https://www.kaggle.com/hugomathien/soccer,The ultimate Soccer database for data analysis...
2,TMDB 5000 Movie Dataset,"Metadata on ~5,000 movies from TMDb",The Movie Database (TMDb),1024,"Version 2,2017-09-28",film,CSV,44 MB,Other,"446,255 views","62,002 downloads","1,394 kernels",46 topics,https://www.kaggle.com/tmdb/tmdb-movie-metadata,Background\nWhat can we say about the success ...
3,Global Terrorism Database,"More than 170,000 terrorist attacks worldwide,...",START Consortium,789,"Version 2,2017-07-19|Version 1,2016-12-08",crime\nterrorism\ninternational relations,CSV,144 MB,Other,"187,877 views","26,309 downloads",608 kernels,11 topics,https://www.kaggle.com/START-UMD/gtd,"Context\nInformation on more than 170,000 Terr..."
4,Bitcoin Historical Data,Bitcoin data at 1-min intervals from select ex...,Zielak,618,"Version 11,2018-01-11|Version 10,2017-11-17|Ve...",history\nfinance,CSV,119 MB,CC4,"146,734 views","16,868 downloads",68 kernels,13 topics,https://www.kaggle.com/mczielinski/bitcoin-his...,Context\nBitcoin is the longest running and mo...


In [16]:
df.shape

(2150, 15)

## Perform Tokenization, Words removal, and Lemmatization

In [17]:
# list for tokenized documents in loop
texts = []

# loop through document list
for i in df['Description'].iteritems():
    # clean and tokenize document string
    raw = str(i[1]).lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [raw for raw in tokens if not raw in en_stop]
    
    # remove stop words from tokens
    stopped_tokens_new = [raw for raw in stopped_tokens if not raw in remove_words]
    
    # lemmatize tokens
    lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in stopped_tokens_new]
    
    # remove word containing only single char
    new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw) == 1]
    
    # add tokens to list
    texts.append(new_lemma_tokens)

# sample data
print(texts[0])

['contains', 'transaction', 'made', 'credit', 'card', 'september', 'european', 'cardholder', 'present', 'transaction', 'occurred', 'two', 'day', 'fraud', 'transaction', 'highly', 'unbalanced', 'positive', 'class', 'fraud', 'account', 'transaction', 'contains', 'numerical', 'input', 'variable', 'result', 'pca', 'transformation', 'unfortunately', 'due', 'confidentiality', 'issue', 'provide', 'original', 'feature', 'background', 'information', 'feature', 'principal', 'component', 'obtained', 'pca', 'feature', 'transformed', 'pca', 'time', 'amount', 'feature', 'time', 'contains', 'second', 'elapsed', 'transaction', 'first', 'transaction', 'feature', 'amount', 'transaction', 'amount', 'feature', 'can', 'used', 'example', 'dependant', 'cost', 'senstive', 'learning', 'feature', 'class', 'response', 'variable', 'take', 'value', 'case', 'fraud', 'otherwise', 'given', 'class', 'imbalance', 'ratio', 'recommend', 'measuring', 'accuracy', 'using', 'area', 'precision', 'recall', 'curve', 'auprc', 'c

## Create term dictionary and document-term matrix

In [18]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [20]:
dictionary

In [22]:
# for k, v in dictionary.items():
#     print(k,v)

## Generate LDA model

I used pre-determined number of topics. It will better calculating perplexity to find the optimum number of topics.    
*top_topics* shows the sorted topics based on the topic coherence.

In [28]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=7, id2word = dictionary, passes=20)
import pprint
pprint.pprint(ldamodel.top_topics(corpus,topn=15))

[([(0.011116325, 'description'),
   (0.0083374484, 'can'),
   (0.0081585506, 'yet'),
   (0.0080767516, 'file'),
   (0.0068363356, 'http'),
   (0.0067467154, 'year'),
   (0.006257317, 'information'),
   (0.0056186169, 'time'),
   (0.0055987849, 'set'),
   (0.005363334, 'csv'),
   (0.0052493629, 'user'),
   (0.0052406141, 'contains'),
   (0.0048864721, 'used'),
   (0.0048376811, 'one'),
   (0.0044507743, 'acknowledgement')],
  -1.1907569607921484),
 ([(0.012243281, 'word'),
   (0.010499313, 'text'),
   (0.010103249, 'can'),
   (0.0079157585, 'price'),
   (0.0058530611, 'contains'),
   (0.0054972009, 'language'),
   (0.0053285342, 'file'),
   (0.0047037825, 'model'),
   (0.0046010995, 'acknowledgement'),
   (0.0045944462, 'corpus'),
   (0.0045317831, 'name'),
   (0.0044834353, 'http'),
   (0.004323517, 'product'),
   (0.003792386, 'value'),
   (0.0037410262, 'using')],
  -1.5700097892459439),
 ([(0.0072528436, 'state'),
   (0.0069877189, 'number'),
   (0.0059843701, 'city'),
   (0.0053042

## Visualize the topic model

Using pyLDAvis, we can create an interactive visualization.

In [27]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/opt/conda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      41.855409        1       1 -0.020338 -0.054940
5      20.754484        1       2 -0.091482 -0.048628
4      11.954082        1       3 -0.071923 -0.074172
0      11.504569        1       4 -0.023281 -0.084835
3       7.940406        1       5 -0.036440 -0.007949
6       3.424515        1       6 -0.065506  0.258359
2       2.566535        1       7  0.308969  0.012165, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
763   Default  1050.000000   university  1050.000000  30.0000  30.0000
742   Default   950.000000        state   950.000000  29.0000  29.0000
1028  Default   514.000000  description   514.000000  28.0000  28.0000
1165  Default   789.000000        model   789.000000  27.0000  27.0000
247   Default   405.000000       player   405.000000  26.0000  26.0000
46    Default   596.000000      feature   596.000000  25.0000  25.0000
100   Default  1439.000000         time  1439.000000  24.0000  24.0000
518   Default   220.000000          yet   220.000000  23.0000  23.0000
143   Default   795.000000       column   795.000000  22.0000  22.0000
1222  Default   523.000000         city   523.000000  21.0000  21.0000
185   Default   399.000000         game   399.000000  20.0000  20.0000
680   Default  1187.000000       number  1187.000000  19.0000  19.0000
1501  Default   620.000000        image   620.000000  18.0000  18.0000
281   Default   393.000000         team   393.000000  17.0000  17.0000
38    Default   283.000000           de   283.000000  16.0000  16.0000
2079  Default   416.000000     language   416.000000  15.0000  15.0000
1387  Default   150.000000      college   150.000000  14.0000  14.0000
796   Default  1033.000000          csv  1033.000000  13.0000  13.0000
462   Default   433.000000     question   433.000000  12.0000  12.0000
3462  Default   435.000000      trained   435.000000  11.0000  11.0000
777   Default   319.000000     activity   319.000000  10.0000  10.0000
931   Default   843.000000         name   843.000000   9.0000   9.0000
51    Default   319.000000        group   319.000000   8.0000   8.0000
338   Default   292.000000      company   292.000000   7.0000   7.0000
2537  Default   147.000000        tweet   147.000000   6.0000   6.0000
1503  Default   165.000000     instance   165.000000   5.0000   5.0000
1071  Default  1220.000000         year  1220.000000   4.0000   4.0000
423   Default   508.000000         many   508.000000   3.0000   3.0000
144   Default   754.000000          com   754.000000   2.0000   2.0000
2118  Default   312.000000      network   312.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
9082   Topic7     8.186708       royale     9.174037   3.5487  -6.8381
5338   Topic7     8.048240         iowa     9.034699   3.5470  -6.8551
5342   Topic7     8.047768        kansa     9.036431   3.5467  -6.8552
1387   Topic7   129.523750      college   150.637786   3.5116  -4.0767
4636   Topic7    53.866314   restaurant    66.788470   3.4476  -4.9541
5428   Topic7    21.308396     southern    27.033617   3.4246  -5.8815
1384   Topic7    59.284683   california    90.144069   3.2436  -4.8582
5361   Topic7    12.274585     missouri    15.046076   3.4590  -6.4330
742    Topic7   188.389978        state   950.758003   2.0439  -3.7021
2395   Topic7    39.132338    institute    95.429213   2.7712  -5.2736
1456   Topic7    31.429863        north    70.297841   2.8576  -5.4928
5468   Topic7    13.628970     virginia    18.378402   3.3636  -6.3284
740    Topic7    18.558676           st    33.011289   3.0867  -6.0196
2876   Topic7    20.344640  nationality    41.145532   2.9583  -5.9277
748    Topic7    27.445146   technology    74.281782   2.6669  -5.6284
5083   Topic7    14.657527     illinois    23.689456   3.1825  